In [8]:
import pickle
import numpy as np
import tensorflow as tf

label_encoder= None
with open('label_encoder.pkl', 'rb') as file:
    label_encoder = pickle.load(file)

model = tf.keras.models.load_model('eegModel.h5')


In [49]:
import os
import pandas as pd
import numpy as np

csvFilesPath = "EEG Data/CSVFiles"
subfolders = [f.path for f in os.scandir(csvFilesPath) if f.is_dir()]
signalLength = 1200
separation = 300

# Lists to store processed data
xVals = []
yVals = []

# Iterate through each subfolder
for folder in subfolders:
    for file in os.scandir(folder):
        if file.is_file() and file.path.endswith('.csv'):
            x = pd.read_csv(file.path)
            y = x['State']
            x = x[['EEG 1', 'EEG 2', 'EEG 3', 'EEG 4', 'EEG 5', 'EEG 6', 'EEG 7', 'EEG 8']].values
            x = np.transpose(x).T

            # Create windows with overlap
            x_windows = [
                x[i: i + signalLength] 
                for i in range(0, x.shape[0] - signalLength + 1, signalLength - separation)
            ]
            
            # Corresponding y windows (you might want to adjust this to take the mode, mean, etc. for each window)
            y_windows = [
                y.iloc[i: i + signalLength].mode()[0] if not y.iloc[i: i + signalLength].empty else None
                for i in range(0, x.shape[0] - signalLength + 1, signalLength - separation)
            ]

            # Ensure only complete windows are included
            x_windows = [win for win in x_windows if win.shape[0] == signalLength]
            y_windows = [label for label in y_windows if label is not None]

            # Stack windows
            try:
                x_windows = np.stack(x_windows)
                y_windows = np.stack(y_windows)
            except ValueError:
                print(f"Warning: Cannot stack windows for file {file.path}. Check dimensions.")
                continue

            # Append to the main lists
            xVals.append(x_windows)
            yVals.append(y_windows)



In [50]:
X = np.vstack(xVals)
y = np.hstack(yVals)
class_mapping = {
    'Select': 'Wait',
    'WarmUp': 'Wait',
    'Rest': 'Wait'
}
# # Convert numpy array to pandas Series to use map
y_series = pd.Series(y)

# # Apply the mapping
y_mapped = y_series.map(class_mapping).fillna(y_series)

# # Convert back to numpy array if needed
y = y_mapped.to_numpy()

print(y)

['Wait' 'Wait' 'Wait' ... 'Wait' 'Wait' 'Left']


In [56]:
X = X.transpose(0,2,1)
print(X.shape)
print(y.shape)

(24893, 8, 1200)
(24893,)


In [69]:
import SignalProcessing.signalProcessing
#Applying bandpass filters
# filtered_input_shape = np.transpose(X, (0, 2, 1))  # New shape: (trials, channels, samples)
X = np.transpose(X,(0,2,1))
X = SignalProcessing.signalProcessing.filter_eeg(X)
X.shape

(24893, 8, 1200)

In [70]:
from tensorflow.keras.utils import to_categorical
y_integer = label_encoder.fit_transform(y)
y_one_hot = to_categorical(y_integer, num_classes=len(label_encoder.classes_))
y_one_hot

array([[0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       ...,
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 1., 0., 0., 0.]])

In [58]:
y_one_hot.shape

(24893, 5)

In [71]:
model.summary()

Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm_2 (LSTM)                   │ (None, 128)            │       680,448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 512)            │        66,048 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 5)              │         1,285 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 879,111 (3.35 MB)

 Trainable params: 879,109 (3.35 MB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [72]:
y_pred = model.predict(X)

778/778 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step


In [73]:
from sklearn.metrics import confusion_matrix,classification_report
confusion_matrix(np.argmax(y_one_hot,axis=1),np.argmax(y_pred,axis=1))

array([[    0,     0,  1500,     0,     0],
       [    0,     0,  1471,     0,     0],
       [    0,     0,  1492,     0,     0],
       [    0,     0,  1495,     0,     0],
       [    0,     0, 18935,     0,     0]], dtype=int64)